In [ ]:
pip install PyMuPDF==1.23.5

Code for test(3) 1.pdf

In [37]:
import fitz
import pandas as pd
import os

def detect_tables(page):

    blocks = page.get_text("dict")["blocks"]
    lines = []
    for block in blocks:
        if "lines" in block:
            for line in block["lines"]:
                for span in line["spans"]:
                    x0, y0, x1, y1 = span["bbox"]
                    text = span["text"].strip()
                    if text:
                        lines.append({"text": text, "x0": x0, "y0": y0})

    rows = {}
    for line in lines:
        y_key = round(line["y0"], -1)
        if y_key not in rows:
            rows[y_key] = []
        rows[y_key].append(line)

    sorted_rows = sorted(rows.items(), key=lambda x: x[0])

    tables = []
    current_table = []
    for y_key, row_lines in sorted_rows:
        row_lines.sort(key=lambda x: x["x0"])
        row_text = [line["text"] for line in row_lines]

        if not current_table or len(row_text) == len(current_table[-1]):
            current_table.append(row_text)
        else:
            if len(current_table) > 1:
                tables.append(current_table)
            current_table = [row_text]

    if len(current_table) > 1:
        tables.append(current_table)

    return tables

def process_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    all_tables = []

    for page_num in range(len(doc)):
        page = doc[page_num]
        tables = detect_tables(page)
        all_tables.extend(tables)

    doc.close()
    return all_tables

def save_to_excel(tables, output_path):
    with pd.ExcelWriter(output_path) as writer:
        for i, table in enumerate(tables):
            df = pd.DataFrame(table)  # Convert table to DataFrame
            df.to_excel(writer, sheet_name=f"Table_{i+1}", index=False, header=False)

def main():

    pdf_path = "test3.pdf"

    if not os.path.exists(pdf_path):
        print(f"Error: {pdf_path} not found.")
        return

    print(f"Processing {pdf_path}...")
    tables = process_pdf(pdf_path)
    if not tables:
        print("No tables detected.")
        return

    output_path = pdf_path.replace(".pdf", "_tables.xlsx")
    save_to_excel(tables, output_path)
    print(f"Tables saved to {output_path}")

if __name__ == "__main__":
    main()

Processing test3.pdf...
Tables saved to test3_tables.xlsx


Code for test6 (1).pdf

In [39]:
import pdfplumber
import pandas as pd
import re
pdf_path = "test3.pdf"


data = []

with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        extracted_text = page.extract_text()

        if extracted_text:

            lines = extracted_text.split("\n")
            for line in lines:

                cleaned_line = re.sub(r"cid\(\d+\)", "", line)


                columns = list(filter(None, cleaned_line.split("  ")))
                data.append(columns)


df = pd.DataFrame(data)


df.to_excel("output.xlsx", index=False, header=False)

print("Data extracted and saved to output.xlsx")

Data extracted and saved to output.xlsx


Decrypting output.xlsx


In [40]:
import pandas as pd
import re


input_file = "output.xlsx"
output_file = "cleaned_transactions.xlsx"

df = pd.read_excel(input_file, header=None)


date_pattern = r"\d{2}-[A-Za-z]{3}-\d{2,4}"
amount_pattern = r"[\d,]+\.\d{2}"
balance_pattern = r"\d{1,3}(?:,\d{3})*\.\d{2}Dr?"


transactions = []


for row in df[0]:
    match = re.findall(date_pattern, str(row))
    if match:
        date = match[0]
        details = row.split(date)[-1].strip()


        transaction_type = "Unknown"
        if "Cash" in details:
            transaction_type = "Cash Deposit"
        elif "IMPS" in details or "UPI" in details:
            transaction_type = "UPI Transfer"
        elif "NEFT" in details:
            transaction_type = "NEFT Transfer"
        elif "Interest" in details or "Int.Coll" in details:
            transaction_type = "Interest Charged"
        elif "Lien Reversal" in details:
            transaction_type = "Lien Reversal"


        amounts = re.findall(amount_pattern, details)
        if len(amounts) == 3:
            debit, credit, balance = amounts
        elif len(amounts) == 2:
            debit, credit, balance = amounts[0], "", amounts[1]
        elif len(amounts) == 1:
            debit, credit, balance = "", "", amounts[0]
        else:
            debit, credit, balance = "", "", ""


        transactions.append([date, transaction_type, details, debit, credit, balance])


columns = ["Date", "Transaction Type", "Details", "Debit (₹)", "Credit (₹)", "Balance (₹)"]
df_cleaned = pd.DataFrame(transactions, columns=columns)


df_cleaned.to_excel(output_file, index=False)

print(f"Structured data saved to {output_file}")

Structured data saved to cleaned_transactions.xlsx
